# Just for Fun
I just wanted to try to make a heat-map of where differences show up for different values. Maybe it isn't useful, but I think it would be a good way to visualize the data.

### Part 1 - Importing the Data

(reusing my path-finding script)

In [2]:
import os
import numpy as np
import matplotlib.pyplot as plt

In [3]:
#doing it this way should find the output_data folder on any OS type or file location
#pathlib
script_path = os.getcwd()
scr_scans_path = os.path.abspath(os.path.join(script_path, "..", "data", "09-08-22_FLIM", "SPC_analyzed", "output_data"))

#define path lists
color_coded_data_paths = []
chi_data_paths = []
photons_data_paths = []

#glob lib
#populate out path lists
for r, d, files in os.walk(scr_scans_path):
    for file in files:
        file_extension = file[-3:]
        if file_extension != "asc":
            continue #skip the non asc files
        full_file_path = os.path.join(scr_scans_path, file)
        if "_color coded value" in file:
            color_coded_data_paths.append(full_file_path)
            continue
        if "_chi" in file:
            chi_data_paths.append(full_file_path)
            continue
        if "_photons" in file:
            photons_data_paths.append(full_file_path)

Making a list of NumPy arrays for each of the SPC outputs. (new script)

In [42]:

#make the empty dicts to fill later
color_coded_data = {}
chi_data = {}
photons_data = {}


def make_np_arrays_from_paths(paths, array_output):
    """We take our found paths of spc output, and we make np arrays from the data
    and put them into a hashmap (dict) using the array_output as the name"""
    for ind, path in enumerate(paths):
        array_name = f"array_numb{ ind}"
        array_output[array_name] = np.genfromtxt(path, delimiter=' ')

make_np_arrays_from_paths(color_coded_data_paths, color_coded_data)
make_np_arrays_from_paths(chi_data_paths, chi_data)
make_np_arrays_from_paths(photons_data_paths, photons_data)

#these arrays are 256 x 256

256


### Part 2 - Math-ing the Arrays to Show Differences
How do we determine the best way to represent the differences and their magnitude in the data....?

My idea is this (I just made this up, idk if it is good or not.)
1. Calculate the average for each point - call this array average_array
2. Subtract each array by the average_array - call these the subtract_array1 etc
3. Multiply all subtract arrays - call this array magnitude_of_difference_array 

##### 2.1 Calculate the average of each point

In [65]:

def make_avg_array(input_data):
    """the average array is what you get if you all all the values individually
    and divide by the total number of arrays you have"""
    first_array = input_data["array_numb0"] #we take the shape of the first array
    avg_array = np.zeros(first_array.shape)
    array_number = 0 #this is bascially a counter of how many arrays there are.
    for k, array in input_data.items(): #loop through and add each point on the array up
        avg_array += array
        array_number += 1

    return avg_array / array_number #divide each point on the array by the total number of arrays


#make the avg_arrays for each
color_avg_array = make_avg_array(color_coded_data)
chi_avg_array = make_avg_array(chi_data)
photons_avg_array = make_avg_array(photons_data)

#print(plt.imshow(color_avg_array))
#print(plt.imshow(chi_avg_array))
#print(plt.imshow(photons_avg_array))


# print(color_avg_array)
# print(np.max(color_avg_array))
# print(plt.imshow(color_avg_array))

#### 2.2 Subtract each array by the average_array